<h2>Upload PDF for book</h2>

In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Saving final sines.pdf to final sines.pdf
Uploaded file: final sines.pdf


<h2>Parsing Data</h2>

1. Converting pdf to csv
2. Removing Special Characters
3. Adding embeddings




In [ ]:
!pip install pdfplumber
import csv
import pdfplumber
import re

def remove_special_characters(string):
    pattern = r'[^a-zA-Z0-9\s\.,!?\n]'  # Pattern to match special characters
    return re.sub(pattern, '', string)

def convert_pdf_to_csv(pdf_path, csv_path):
    with pdfplumber.open(pdf_path) as pdf:
        with open(csv_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file,escapechar="\\")
            writer.writerow(['Page Number', 'Page Text'])

            for page_number, page in enumerate(pdf.pages, start=1):
                page_text = remove_special_characters(page.extract_text())
                writer.writerow([page_number, page_text])

#file_name= next(iter(uploaded))

# Provide the paths for the PDF and CSV files
pdf_file_path = 'final sines.pdf'
csv_file_path = 'finalsines.csv'

# Call the function to convert PDF to CSV
convert_pdf_to_csv(pdf_file_path, csv_file_path)

<h3>Visualize the CSV file</h3>

In [ ]:
import pandas as pd
dataset = pd.read_csv('finalsines.csv')
dataset.head()


,Page Number,Page Text
0,1,Contact Directory\nContact Directory for Schoo...
1,2,SINES Faculty Staff Directory\nPrincipal of SI...
2,3,Principal of SINES Syed Irtiza Ali Shah\nPubl...
3,4,Project Director NUST Chip Design Center NCDC ...
4,5,Head Of Department HOD ENGINEERING Mian Ilyas ...


<h2>Getting embeddings for all pages</h2>

In [ ]:
!pip install --upgrade pip
!pip install openai
!pip install typing_extensions==4.7.1
!pip install typing_extensions --upgrade
!pip install openai --upgrade
!pip install requests
!pip install typing-extensions --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and pl

In [ ]:
from typing_extensions import Annotated, deprecated

import openai
EMBEDDINGS_MODEL = "text-embedding-3-small"


In [ ]:
from openai import OpenAI
client = OpenAI(api_key='openai key')

def get_embeddings(input):
  response = client.embeddings.create(
      input=input,
      model="text-embedding-3-small"
  )

  return response.data[0].embedding

In [ ]:
import time
embeddings = []
run_count = 0
total = len(dataset)
while run_count <= total :
    for idx, row in enumerate(zip(dataset['Page Number'], dataset['Page Text'])):
        result = get_embeddings(row[1])
        embeddings.append(result)
        print(f'Progress : {len(embeddings)}/{total}')
        run_count += 1
        if run_count >= total:
          break
          #for instance max requests per minute allowed = 60 so after 50 requests we wait for 60s
        if run_count % 25 == 0 :
              time.sleep(20)


Progress : 1/65
Progress : 2/65
Progress : 3/65
Progress : 4/65
Progress : 5/65
Progress : 6/65
Progress : 7/65
Progress : 8/65
Progress : 9/65
Progress : 10/65
Progress : 11/65
Progress : 12/65
Progress : 13/65
Progress : 14/65
Progress : 15/65
Progress : 16/65
Progress : 17/65
Progress : 18/65
Progress : 19/65
Progress : 20/65
Progress : 21/65
Progress : 22/65
Progress : 23/65
Progress : 24/65
Progress : 25/65
Progress : 26/65
Progress : 27/65
Progress : 28/65
Progress : 29/65
Progress : 30/65
Progress : 31/65
Progress : 32/65
Progress : 33/65
Progress : 34/65
Progress : 35/65
Progress : 36/65
Progress : 37/65
Progress : 38/65
Progress : 39/65
Progress : 40/65
Progress : 41/65
Progress : 42/65
Progress : 43/65
Progress : 44/65
Progress : 45/65
Progress : 46/65
Progress : 47/65
Progress : 48/65
Progress : 49/65
Progress : 50/65
Progress : 51/65
Progress : 52/65
Progress : 53/65
Progress : 54/65
Progress : 55/65
Progress : 56/65
Progress : 57/65
Progress : 58/65
Progress : 59/65
Progre

<h2>Adding the embeddings to the csv file</h3>

We store the embeddings in the csv file so that we dont have to make requests again and again for the embeddings

Save embeddings to the csv

In [ ]:
import csv
import tempfile
import shutil

csv_file = 'finalsines.csv'
temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False)


with open(csv_file, 'r') as file_in, open(temp_file.name, 'w', newline='') as file_out:
    reader = csv.reader(file_in)
    writer = csv.writer(file_out)

    header = next(reader)
    header.append('Embeddings')
    writer.writerow(header)  # Write the modified header to the temporary file

    for row in reader:
        embedding_value = embeddings.pop(0)  # Get the next embedding value from the list
        row.append(embedding_value)  # Append the embedding value to the row
        writer.writerow(row)  # Write the modified row to the temporary file

shutil.move(temp_file.name, csv_file)


'finalsines.csv'

<h2>Updated CSV file</h2>

In [ ]:
import pandas as pd
dataset = pd.read_csv('finalsines.csv')
dataset.head()

,Page Number,Page Text,Embeddings
0,1,Contact Directory\nContact Directory for Schoo...,"[-0.022668220102787018, -0.06522712856531143, ..."
1,2,SINES Faculty Staff Directory\nPrincipal of SI...,"[0.04392107576131821, 0.003571220673620701, 0...."
2,3,Principal of SINES Syed Irtiza Ali Shah\nPubl...,"[0.009437339380383492, -0.009277619421482086, ..."
3,4,Project Director NUST Chip Design Center NCDC ...,"[0.009087448939681053, -0.06644821166992188, 0..."
4,5,Head Of Department HOD ENGINEERING Mian Ilyas ...,"[-0.03978569805622101, -0.039672862738370895, ..."
